In [1]:
from pyomo.environ import *
import pandas as pd
import numpy as np
from ipynb.fs.full.ss_radial_distribution_model import optimization_model
from ipynb.fs.full.data_vis import vis

In [2]:
def read_input(filename):
    SystemDemand = pd.read_excel(filename, sheet_name = 'SystemDemand')
    NodeData = pd.read_excel(filename, sheet_name='NodeData')
    LineData = pd.read_excel(filename, sheet_name='LineData')

    return {'SystemDemand':SystemDemand, 'NodeData':NodeData, 'LineData':LineData}

In [3]:
data = read_input(r'Input_Files\InputData34.xlsx')
for i in data.keys():
    print(i, ':\n', data[i])
    print('\n')

Sbase = 1000 # kVA
Vnom = 11/np.sqrt(3)    # kV
Zbase = (Vnom**2)*1000/Sbase # Ohm
Vmax_pow = 1.05
Vmin = 0.8
model, buses, lines, Pd, Qd = optimization_model(data, Vmax_pow, Vmin, Vnom, Sbase, Zbase)

SystemDemand :
    TIME   PD   QD
0     1  1.0  1.0
1     2  1.2  1.2
2     3  1.4  1.4
3     4  0.8  0.8


NodeData :
     NODES  Tb     PD    QD
0       1   1    0.0   0.0
1       2   0  130.3  80.7
2       3   0    0.0   0.0
3       4   0  130.3  80.7
4       5   0  130.3  80.7
5       6   0    0.0   0.0
6       7   0    0.0   0.0
7       8   0  130.3  80.7
8       9   0  130.3  80.7
9      10   0    0.0   0.0
10     11   0  130.3  80.7
11     12   0   77.6  47.6
12     13   0   40.8  25.5
13     14   0   40.8  25.5
14     15   0   40.8  25.5
15     16   0    7.6   4.2
16     17   0  130.3  80.7
17     18   0  130.3  80.7
18     19   0  130.3  80.7
19     20   0  130.3  80.7
20     21   0  130.3  80.7
21     22   0  130.3  80.7
22     23   0  130.3  80.7
23     24   0  130.3  80.7
24     25   0  130.3  80.7
25     26   0  130.3  80.7
26     27   0   77.6  48.1
27     28   0   42.5  27.2
28     29   0   42.5  27.2
29     30   0   42.5  27.2
30     31   0   32.3  21.2
31     32   0   

In [4]:
solver = SolverFactory('ipopt')
solver.solve(model, tee=True)

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     1984
Number of nonzeros in inequality constraint Jacobian.:      264
Number of nonzeros in Lagrangian Hessian.............:      660

Total number of variables............................:      536
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Tot

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 804, 'Number of variables': 536, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.12.13\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.18748211860656738}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [5]:
# CHECK LOSSES+DEMAND=SUPPLY
losses = value(model.obj)
supply = sum([sum(value(model.Ps[n, t]) for n in model.NODES) for t in model.TIME])
demand = sum([sum(value(model.Pd[n, t]) for n in model.NODES) for t in model.TIME])
print([supply, demand, losses])
print([supply*Sbase, demand*Sbase, losses*Sbase])

[12.765194771926799, 11.557480000000004, 1.2077147719268]
[12765.1947719268, 11557.480000000003, 1207.7147719268]


In [6]:
vis(data, buses, lines, model, Sbase)

NameError: name 'lines' is not defined